# Plot customizations

<b>Objective</b>

This vignette contains various snippets of code<br>
that show how plots and data can be customized<br>
to ones requirements.

The h5 file used in this notebook can be found [here](https://github.com/MissionBio/mosaic-jupyter/tree/master/customizations)

In [ ]:
import missionbio.mosaic as ms

sample = ms.load_example_dataset('3 cell mix')

All the interactive plotting functions return a [plotly figure](https://plotly.com/python/figure-structure/). In case the layout or the color<br>
scheme is not suitable for your data type, they can be changed before creating the final figure.<br>

1. The color for the plots are store either in the individual traces or the layout
   attributes of the plotly figure.
2. Mosaic also contains a list of colors that can be used to customize the plots.
3. Most importantly the mosaic configuration can be modified for certain layers
   of the assays, and that will affect the color scale in all plots for those layers

In [ ]:
# Plot the first few colors
import seaborn as sns
sns.palplot(ms.COLORS[:21])

### Configuration options

In [ ]:
# The configuration of the colorscales is stored in ms.Config.Colorscale

ms.Config.Colorscale

In [ ]:
# The default values for each assay are stored in the respective options

ms.Config.Colorscale.Dna.NGT  # This is the default colorscale for the DNA NGT layer in all plots

### Updating the colors for plots

#### Colors of the values

In [ ]:
# The default color scale for NGT is a monochromatic blue color scheme.

sample.dna.heatmap('NGT')

In [ ]:
# Assuming these are new desired colors
# NGT=0 (WT) - grey
# NGT=1 (HET) - brown
# NGT=2 (HOM) - purple
# NGT=3 (missing) - black

wt_col = ms.COLORS[9]
het_col = ms.COLORS[5]
hom_col = ms.COLORS[4]
miss_col = ms.COLORS[-1]

sns.palplot([wt_col, het_col, hom_col, miss_col])

In [ ]:
# Update the coloraxis to make a plot with the new colors

ms.Config.Colorscale.Dna.NGT = [
    (0 / 4, wt_col), (1 / 4, wt_col),
    (1 / 4, het_col), (2 / 4, het_col),
    (2 / 4, hom_col), (3 / 4, hom_col),
    (3 / 4, miss_col), (4 / 4, miss_col)
]

In [ ]:
# All subsequent plots using NGT will use the above colorscale

sample.dna.heatmap("NGT")

In [ ]:
# The same method can be used to update scatterplot which are colored by NGT

sample.dna.scatterplot('umap', colorby='NGT', features=sample.dna.ids()[:4])

#### Colors of the labels

Now the colors in the heatmap conflict with the colors in the labels. To customize those, the palette can be changed

In [ ]:
# This is the current palette

sample.dna.get_palette()

In [ ]:
# Update this palette. It is not required to use the built in colors
# Any hexadecimal colors can be passed.

new_palette = {
    'Jurkat': ms.COLORS[3],
    'KG-1': ms.COLORS[4],
    'Mixed': '#c7c7c7',  # Use hexadecimal colors
    'TOM-1': ms.COLORS[5]
}


sample.dna.set_palette(new_palette)

In [ ]:
# Make the heatmap with the new colors

sample.dna.heatmap('NGT')

### Scaling the plots

Often the cnv heatmaps contain too many genes or amplicons to fit in the default layout.<br>
This is usually not an issue when they are interactive, but when exporting as static images<br>
it hinder the ability to interpret them.

Plotly provides an option to convert interactive figures to static images

In [ ]:
# Scale the figure width and plot as a static image.
# Double click on the plot to zoom-in and improve the resolution

import missionbio.mosaic.utils as mutils

fig = sample.cnv.heatmap('ploidy', features='genes')
fig.layout.width = 1600
fig.show("jpg")

### Smoothening the values

In case the color scale get skewed to high poidy, a max value can be imposed to generate a more interpretable heatmap

The colorscale can also be changed as desired. A list of color scales can be found in the [plotly documentation](https://plotly.com/python/builtin-colorscales/)

In [ ]:
# The plots can also be smoothed using a moving average with the convolve parameter

sample.cnv.heatmap('ploidy', features='genes', convolve=5)

### Filtering the data

Often the number of amplicons in CNV might take over the sample level heatmap making the plot uninterpretable. Moreover there might be certain non-differentiating variants and protein in the panel. These can be dropped before making the final heatmap.

In [ ]:
# The genes to plot on the sample heatmap

genes = ['EZH2', 'TET2']

In [ ]:
# When `features` is None, all the ids are clustered and plotted
fig = sample.heatmap(
    ("dna", "protein", "cnv"),
    attributes=("NGT", "normalized_counts", "ploidy"),
    features=(None, None, genes)
)

# Update the width of the plot [See the section on CNV heatmaps]
fig.layout.width = 1600

# Show a static plot
fig.show("jpg")

### Resetting the configuration

<b>Notice that the DNA colorscale is still the same as the one set in the configuration</b>

All configuration values can be reset using the `reset` method.

In [ ]:
ms.Config.Colorscale.Dna.reset("NGT")  # This will reset the NGT colorscale

# To recursively reset all colorscales of the Dna assay
# run the `reset` function of the `Colorscale` option.
ms.Config.Colorscale.reset("Dna")

ms.Config.Colorscale.reset()  # By skipping the parameter, all values will be reset i.e. Dna, Cnv, and Protein

In [ ]:
# Creating the same plot again will 
fig = sample.heatmap(
    ("dna", "protein", "cnv"),
    attributes=("NGT", "normalized_counts", "ploidy"),
    features=(None, None, genes)
)

# Update the width of the plot [See the section on CNV heatmaps]
fig.layout.width = 1600

# Show a static plot
fig.show("jpg")

### Saving the plots

All plotly figures can be saved as .jpg, .png, .svg, and .webp formats

In [ ]:
fig = sample.dna.heatmap("NGT")

fig.write_image("./saved_image.pdf")  # Save as a vector .pdf file
fig.write_image("./saved_image.svg")  # Save as a vector .svg file
fig.write_image("./saved_image.jpg")  # Save as a .jpeg
fig.write_image("./saved_image.png")  # Save as a .png
fig.write_image("./saved_image.webp")  # Save as a .webp